In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
PROJ_PATH = '/content/drive/My Drive/yolo-v4-tf.keras'
PROJ_PATH = '.'
import sys
sys.path.append(PROJ_PATH)

In [ ]:
import cv2
from glob import glob
import numpy as np
from utils import DataGenerator, preprocess_true_boxes
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import math
import os
from models import Yolov4, yolov4_head, get_boxes, nms
from config import yolo_config
from loss import *
print(tf.__version__)

In [ ]:
with open(os.path.join(PROJ_PATH, 'dataset/train_txt/anno2.txt')) as f:
    lines = f.readlines()
lines = lines[:]
val_lines = lines[:16]

NUM_CLASS = 3
BS = 8
anchors = np.array([12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]).reshape((-1, 2))

data_gen = DataGenerator(lines[:], BS, (416, 416), num_classes=NUM_CLASS, folder_path=PROJ_PATH, anchors=anchors)
data_gen_val = DataGenerator(val_lines[:], BS, (416, 416), num_classes=NUM_CLASS, folder_path=PROJ_PATH, anchors=anchors)

model = Yolov4(
                weight_path=None,
                class_name_path=os.path.join(PROJ_PATH, 'bccd_classes.txt')
              )
model.build_model(load_pretrained=False)

print('num class : ', model.num_classes)

In [ ]:
model.yolo_model.load_weights(os.path.join(PROJ_PATH, 'bccd.h5'), by_name=True)

In [ ]:
# model.predict('/content/drive/My Drive/yolo-v4-tf.keras/dataset/train_img2/test3.jpg')

In [ ]:
# i = np.random.randint(0, 300)
# path = f'/content/drive/My Drive/yolo-v4-tf.keras/dataset/train_img/BloodImage_00000.jpg'
# print(path)
# model.predict(path)

In [ ]:
# eye 86 274 146 305
# eye 214 273 282 303
# nose 138 334 220 385
# mouth 139 404 220 437
model.class_names

#### Generate GT .txt
eye 86 274 146 305

mouth 139 404 220 437

In [ ]:
GT_PATH = os.path.join(PROJ_PATH, 'eval/ground_truth')
with open(os.path.join(PROJ_PATH, 'dataset/train_txt/anno2.txt')) as file:
    for line in file:
        line = line.split(' ')
        filename = line[0].split(os.sep)[-1].split('.')[0]
        objs = line[1:]
        # export txt file
        with open(os.path.join(GT_PATH, filename+'.txt'), 'w') as output_file:
            for obj in objs:
                x_min, y_min, x_max, y_max, class_id = [float(o) for o in obj.strip().split(',')]
                output_file.write(f'{model.class_names[int(class_id)]} {x_min} {y_min} {x_max} {y_max}\n')

#### generate pred .txt

In [ ]:
model.export_prediction(os.path.join(PROJ_PATH, 'dataset/train_txt/anno2.txt'), 
                        os.path.join(PROJ_PATH, 'eval/pred_result'),
                        os.path.join(PROJ_PATH, 'dataset/train_img'))

In [ ]:
gt_paths = glob(os.path.join(GT_PATH, '*.txt'))
len(gt_paths)

In [ ]:
model.eval_map(gt_folder_path = os.path.join(PROJ_PATH, 'eval/ground_truth'), 
               pred_folder_path= os.path.join(PROJ_PATH, 'eval/pred_result'), 
               temp_json_folder_path= os.path.join(PROJ_PATH, 'eval/json'),
              output_files_path=os.path.join(PROJ_PATH, 'eval/result'))

In [ ]:
glob(os.path.join(PROJ_PATH, 'eval/pred_result', '*.txt'))